In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from scipy.stats import beta as sp_beta

import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')
plt.rcParams['figure.figsize'] = 20, 15
plt.rcParams['font.size'] = 20
plt.rcParams['lines.linewidth'] = 2

import sys
sys.path.insert(0, '../../..')
import assignment
import leximin_assignment
import matrix_gen
import kde_utils

from tqdm import tqdm

import warnings

In [2]:
N = 30
N_INTVS = 5
N_EXPERIMENTS = 500  # 10 and 500

CAPACITIES = np.ones((N_INTVS,), dtype=int) * (N // N_INTVS)

In [3]:
distributions = [
    np.random.uniform,
    np.random.beta,
    np.random.beta,
    np.random.beta,
    np.random.beta,
    np.random.beta,
    np.random.beta,
    np.random.beta
]

dist_params = [
    (0, 1),
    (1, 6),
    (6, 1),
    (2, 5),
    (5, 2),
    (0.5, 0.5),
    (0.6, 0.1),
    (0.1, 0.6),
]

matrix_generators = [
    matrix_gen.MatrixGenerator(dist, params, N, N_INTVS)
    for dist, params in zip(distributions, dist_params)
]

labels = [
    f'{dist.__name__}({params[0]}, {params[1]})'
    for dist, params in zip(distributions, dist_params)
]
labels

['uniform(0, 1)',
 'beta(1, 6)',
 'beta(6, 1)',
 'beta(2, 5)',
 'beta(5, 2)',
 'beta(0.5, 0.5)',
 'beta(0.6, 0.1)',
 'beta(0.1, 0.6)']

# Sorted vs. not sorted

In [4]:
columns = []
for label in labels:
    columns.append(label)
    columns.append(label + '_sorted')

pof_df = pd.DataFrame(columns=columns)

In [5]:
# for i in tqdm(range(N_EXPERIMENTS)):
while pof_df.shape[0] < N_EXPERIMENTS:
    print(pof_df.shape[0])
    
    temp_row = []
    move_on = False
    
    for matrix_generator in matrix_generators:
        if move_on:
            break
        
        (matrix, sorted_matrix), (increase_matrix, sorted_increase_matrix) = matrix_generator.get_increase_matrix_pair()
        
#         _, unique_counts = np.unique(increase_matrix[increase_matrix != 0], return_counts=True)
#         if np.any(unique_counts > 1):
#             move_on = True
#             print('Repeated')
#             # continue
#             raise TypeError
        
        ### Original matrix
        # Efficient assignment
        eff_assigner = assignment.AssignmentHelperV2(
            increase_matrix, CAPACITIES
        )
        eff_assignments = eff_assigner.ip_solve()
        eff_cost = eff_assigner.get_cost(
            eff_assignments, cost_matrix=matrix
        )
        
        # Leximin assignment
        lex_assigner = leximin_assignment.LeximinAssignmentHelperV3(
            increase_matrix * 100, CAPACITIES
        )
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            lex_assignments = lex_assigner.solve()
        
        ### Code-breaking error - skip if encounter for now
        if lex_assignments is False:
            move_on = True
            print('Other')
            continue
        
        lex_cost = lex_assigner.get_cost(
            lex_assignments, cost_matrix=matrix
        )
        
        temp_row.append(lex_cost / eff_cost)
        
        ### Row-sorted matrix
        # Efficient assignment
        eff_assigner = assignment.AssignmentHelperV2(
            sorted_increase_matrix, CAPACITIES
        )
        eff_assignments = eff_assigner.ip_solve()
        eff_cost = eff_assigner.get_cost(
            eff_assignments, cost_matrix=sorted_matrix
        )
        
        # Leximin_assignment
        lex_assigner = leximin_assignment.LeximinAssignmentHelperV3(
            sorted_increase_matrix * 100, CAPACITIES
        )
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            lex_assignments = lex_assigner.solve()
            
        ### Code-breaking error - skip if encounter for now
        if lex_assignments is False:
            move_on = True
            print('Other')
            continue
             
        lex_cost = eff_assigner.get_cost(
            lex_assignments, cost_matrix=sorted_matrix
        )
        
        temp_row.append(lex_cost / eff_cost)
    
    if move_on:
        continue
    pof_df.loc[pof_df.shape[0]] = temp_row

0
1
2
3
4
Other
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
Other
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
Other
43
Other
43
Other
43
44
Other
44
45
Other
45
46
47
Other
47
48
Other
48
Other
48
49
50
51
52
53
54
Other
54
55
Other
55
56
57
58
59
60
61
62
63
64
65
66
67
68
Other
68
Other
68
Other
68
69
70
71
72
73
74
Other
74
75
76
Other
76
77
78
79
80
81
82
83
84
85
Other
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
Other
119
120
121
122
123
124
Other
124
125
126
127
128
129
130
131
132
133
Other
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
Other
154
155
156
157
158
159
Other
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
Other
175
Other
175
176
177
178
179
Other
179
180
181
182
Other
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
Other
200
201
202
203
204
Other
204
205
Other
205
206
2

In [6]:
print(pof_df.shape)
pof_df.head()

(500, 16)


,"uniform(0, 1)","uniform(0, 1)_sorted","beta(1, 6)","beta(1, 6)_sorted","beta(6, 1)","beta(6, 1)_sorted","beta(2, 5)","beta(2, 5)_sorted","beta(5, 2)","beta(5, 2)_sorted","beta(0.5, 0.5)","beta(0.5, 0.5)_sorted","beta(0.6, 0.1)","beta(0.6, 0.1)_sorted","beta(0.1, 0.6)","beta(0.1, 0.6)_sorted"
0,1.008206,1.044019,1.005459,1.046464,1.000000,1.018569,1.03055,1.027236,1.000000,1.028134,1.017451,1.059270,1.000000,1.025862,1.000000,1.000000
1,1.014315,1.083174,1.002986,1.071895,1.002592,1.022789,1.00000,1.043394,1.001829,1.046792,1.000000,1.035171,1.004635,1.023892,1.000000,1.000000
2,1.001647,1.015748,1.000229,1.011592,1.000806,1.012952,1.00000,1.029004,1.000000,1.008398,1.000000,1.052994,1.012260,1.025020,1.000000,1.000000
3,1.005338,1.029156,1.005972,1.008441,1.000000,1.017438,1.00000,1.021022,1.000000,1.008951,1.000000,1.043385,1.000000,1.030679,1.000000,1.000000
4,1.025598,1.036391,1.013452,1.028359,1.000000,1.019482,1.00000,1.032147,1.000000,1.016615,1.000000,1.035351,1.000000,1.016476,1.001645,1.038469


In [7]:
pof_df.to_csv('pof_df.csv', index=False)


# Sorted rows + many agents

In [ ]:
N = 100
N_INTVS = 5
N_EXPERIMENTS = 500  # 10 and 500

CAPACITIES = np.ones((N_INTVS,), dtype=int) * (N // N_INTVS)
sorted_pof_df = pd.DataFrame(columns=labels)

In [ ]:
while sorted_pof_df.shape[0] < N_EXPERIMENTS:
    print(sorted_pof_df.shape[0])
    
    temp_row = []
    move_on = False
    
    for matrix_generator in matrix_generators:
        if move_on:
            break
        
        (_, sorted_matrix), (__, sorted_increase_matrix) = matrix_generator.get_increase_matrix_pair(round_=False)
        
        _, unique_counts = np.unique(increase_matrix[increase_matrix != 0], return_counts=True)
        if np.any(unique_counts > 1):
            move_on = True
            print('Repeated')
            continue
            # raise TypeError
        
        # Efficient assignment
        eff_assigner = assignment.AssignmentHelperV2(
            sorted_increase_matrix, CAPACITIES
        )
        eff_assignments = eff_assigner.ip_solve()
        eff_cost = eff_assigner.get_cost(
            eff_assignments, cost_matrix=sorted_matrix
        )
        
        # Leximin_assignment
        lex_assigner = leximin_assignment.RowSortedLeximinAssignmentHelper(
            sorted_increase_matrix, CAPACITIES
        )
        lex_assignments = lex_assigner.solve()
        lex_cost = eff_assigner.get_cost(
            lex_assignments, cost_matrix=sorted_matrix
        )
        
        temp_row.append(lex_cost / eff_cost)
    
    if move_on:
        continue
    sorted_pof_df.loc[sorted_pof_df.shape[0]] = temp_row